In [1]:
import os
import time 
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=MY_ENV_VAR)
print("client ready")

client ready


In [2]:
System_Prompt_question_gen = "Below is the user's goal or task, based on clear understanding give the following in JSON:\n- List of top(max 5) very important questions for the user with options to improve the goal statement and make the goal less vague. Questions to be asked to remove vagueness and improvement for more clarity for others. Its type can be \"choice\",  \"input\" or \"date\", if input then give options as empty list(but prefer choice over input). Do not ask questions only if it's very well described(respond with empty list for questions). \n- State weather if it is a product, service or invalid goal (in goal_type). Format - {\"questions\":[{\"question\":\"\",\"type\":\"\",\"options\":[\"\",\"\"],},{}],\"goal_type\":\"\"}"

question_gen_few_shot = [ {
                    "role": "user",
                    "content": "Location: Oakland, CA;\nGoal: I want a new car",
                },
                {
                    "role": "system",
                    "content": "{\"questions\":[{\"question\":\"What type of car do you want?\",\"type\":\"choice\",\"options\":[\"SUV\", \"Sedan\", \"Truck\", \"Coupe\"]},{\"question\":\"What is the budget for the car?\",\"type\":\"choice\",\"options\":[\"$1000\", \"$2000\", \"$3000\", \"$4000\"]},{\"question\":\"Which brand of car do you prefer?\",\"type\":\"choice\",\"options\":[\"Toyota\", \"Honda\", \"Ford\", \"Chevrolet\"]}],\"goal_type\":\"product\"}",
                },
                {
                    "role": "user",
                    "content": "Goal:I want to do an internship in UC Davis in the fiels of Microbiology.",
                },
                { "role": "system",
                 "content": "{\"questions\":[{\"question\":\"What is the duration of the internship?\",\"type\":\"choice\",\"options\":[\"1 month\", \"2 months\", \"3 months\", \"4 months\"]},{\"question\":\"What type of internship do you want?\",\"type\":\"choice\",\"options\":[\"Paid\",\"Un-Paid\"]},{\"question\":\"What is the field of study?\",\"type\":\"input\",\"options\":[]}],\"goal_type\":\"service\"}"
                }
                ]

In [3]:
"{\"questions\":[{\"question\":\"What type of car do you want?\",\"type\":\"choice\",\"options\":[\"SUV\", \"Sedan\", \"Truck\", \"Coupe\"]},{\"question\":\"What is the budget for the car?\",\"type\":\"choice\",\"options\":[\"$1000\", \"$2000\", \"$3000\", \"$4000\"]},{\"question\":\"Which brand of car do you prefer?\",\"type\":\"choice\",\"options\":[\"Toyota\", \"Honda\", \"Ford\", \"Chevrolet\"]}],\"goal_type\":\"product\"}"

'{"questions":[{"question":"What type of car do you want?","type":"choice","options":["SUV", "Sedan", "Truck", "Coupe"]},{"question":"What is the budget for the car?","type":"choice","options":["$1000", "$2000", "$3000", "$4000"]},{"question":"Which brand of car do you prefer?","type":"choice","options":["Toyota", "Honda", "Ford", "Chevrolet"]}],"goal_type":"product"}'

In [4]:
GOAL = "Want a chef for an after wedding party"

In [5]:

start_time = time.time()

response = client.chat.completions.create(
  model= "gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": System_Prompt_question_gen},
    *question_gen_few_shot,
    {"role": "user", "content": f"Goal:{GOAL}"},
  ]
)

end_time = time.time()
print(response)

print(f'Total Tokens:{response.usage.total_tokens}, Prompt Token {response.usage.prompt_tokens}, Completion Token {response.usage.completion_tokens}')
print(f'Time Taken: {end_time - start_time} Sec\n')

response = json.loads(response.choices[0].message.content)
print(f"Goal: {GOAL}")
print(json.dumps(response, indent=4))

ChatCompletion(id='chatcmpl-907qMUTAvGkU9kKGscfMa99tgzbYg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"questions":[{"question":"How many guests are expected at the after wedding party?","type":"choice","options":["Less than 50","50-100","100-150","More than 150"]},{"question":"What type of cuisine do you prefer for the party?","type":"input","options":[]},{"question":"Do you have any specific dietary restrictions or special requests for the chef to consider?","type":"input","options":[]}],"goal_type":"service"}', role='assistant', function_call=None, tool_calls=None))], created=1709817178, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=96, prompt_tokens=409, total_tokens=505))
Total Tokens:505, Prompt Token 409, Completion Token 96
Time Taken: 3.1263108253479004 Sec

Goal: Want a chef for an after wedding party
{
    "questions": [
        {
   

In [6]:

improved_goal_answers = "Q/A: \n"

# ask user the  with options
print("Asking user the questions")
for question in response['questions']:
    print(f"\nQuestion: {question['question']}")
    improved_goal_answers += f" {question['question']}"
    if question['type'] == "choice":
        print(f"Options: {question['options']}")
    print(f"Type: {question['type']}")
    value = input(" Answer: ")
    improved_goal_answers += f" -> {value}\n"
    print("")

print(improved_goal_answers)


Asking user the questions

Question: How many guests are expected at the after wedding party?
Options: ['Less than 50', '50-100', '100-150', 'More than 150']
Type: choice


Question: What type of cuisine do you prefer for the party?
Type: input


Question: Do you have any specific dietary restrictions or special requests for the chef to consider?
Type: input

Q/A: 
 How many guests are expected at the after wedding party? -> 50-100
 What type of cuisine do you prefer for the party? -> Indian, Europian
 Do you have any specific dietary restrictions or special requests for the chef to consider? -> no



In [7]:
question_merge_sys_prompt = "Given the user's goal and the questions asked to the user with its answers, merge the questions into the goal to make it less vague. If the goal is already well described, respond with the goal as it is. Respond in JSON, Format - {\"merged_goal\":\"\"}"

question_merge_few_shot = [ {
                    "role": "user",
                    "content": "Goal: I want a new car,  Answers: What type of car do you want? - SUV, Which brand of car do you prefer? - Toyota, Location: Oakland, CA",
                },
                {
                    "role": "system",
                    "content": "{\"merged_goal\":\"I want a new Toyota SUV car in Oakland, CA\"}",
                }]

In [8]:

start_time = time.time()

response = client.chat.completions.create(
  model= "gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": question_merge_sys_prompt},
    *question_merge_few_shot,
    {"role": "user", "content": f"Goal:{GOAL},{improved_goal_answers}"},
  ]
)

end_time = time.time()
print(response)

print(f'Total Tokens:{response.usage.total_tokens}, Prompt Token {response.usage.prompt_tokens}, Completion Token {response.usage.completion_tokens}')
print(f'Time Taken: {end_time - start_time} Sec\n')

response = json.loads(response.choices[0].message.content)
print(f"Goal: {GOAL}")
print(json.dumps(response, indent=4))

ChatCompletion(id='chatcmpl-907rtQTZodTknWi9mY4Yluc9R5Lmx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"merged_goal":"Want an Indian or European cuisine chef for an after wedding party with 50-100 guests and no specific dietary restrictions in mind"}', role='assistant', function_call=None, tool_calls=None))], created=1709817273, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=30, prompt_tokens=195, total_tokens=225))
Total Tokens:225, Prompt Token 195, Completion Token 30
Time Taken: 1.5962140560150146 Sec

Goal: Want a chef for an after wedding party
{
    "merged_goal": "Want an Indian or European cuisine chef for an after wedding party with 50-100 guests and no specific dietary restrictions in mind"
}
